In [45]:
import torch
import torch.nn as nn
import torchvision
from torch.utils import data
import numpy as np
# import jittor

In [46]:
#导入数据并且进行修改
dataset=torchvision.datasets.MNIST("mnist_data",train=True,download=True,
                                   transform=torchvision.transforms.Compose([
                                       torchvision.transforms.Resize(28),
                                       torchvision.transforms.ToTensor(),
                                       torchvision.transforms.Normalize(mean=[0.5],std=[0.5])]))
print("数据长度为:",len(dataset))
for i in range(6):
    print(dataset[i][0].shape)#0是样本的(通道,h,w),1是样本标签


数据长度为: 60000
torch.Size([1, 28, 28])
torch.Size([1, 28, 28])
torch.Size([1, 28, 28])
torch.Size([1, 28, 28])
torch.Size([1, 28, 28])
torch.Size([1, 28, 28])


In [47]:
image_size=[1,28,28]
latent_dim = 96

In [48]:
class Generator(nn.Module):
    def __init__(self) -> None:
        super(Generator,self).__init__()
        self.model=nn.Sequential(
            nn.Linear(latent_dim ,64),
            nn.ReLU(inplace=True),
            nn.Linear(64,128),
            nn.ReLU(inplace=True),
            nn.Linear(128,256),
            nn.ReLU(inplace=True),
            nn.Linear(256,512),
            nn.ReLU(inplace=True),
            nn.Linear(512,1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024,np.prod(image_size, dtype=np.int32)),
            nn.Tanh()
        )
    

    def forward(self,z):
        #shape of z,[batchsize,1*28*28],随机生成的高斯噪声,维度可以是任意的
        output=self.model(z)
        img=output.reshape(z.shape[0],*image_size)
        return img
        
    pass

In [49]:
torch.prod(torch.tensor(image_size),dtype=torch.int32)

tensor(784, dtype=torch.int32)

In [50]:
class Discriminator(nn.Module):
    def __init__(self) -> None:
        super(Discriminator,self).__init__()
        self.model=nn.Sequential(
            nn.Linear(np.prod(image_size, dtype=np.int32),1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024,512),
            nn.ReLU(inplace=True),
            nn.Linear(512,256),
            nn.ReLU(inplace=True),
            nn.Linear(256,128),
            nn.ReLU(inplace=True),
            nn.Linear(128,1),
            nn.Sigmoid()
        )
    
    def forward(self,img):
        #shape of img,[batchsize,1,28,28],随机生成的高斯噪声
        p=self.model(img.reshape(img.shape[0],-1))
        return p


In [51]:
#Tranining
batchsize=32
dataloader=data.DataLoader(dataset,batch_size=batchsize,shuffle=True)
generator=Generator()
g_optimizer=torch.optim.Adam(generator.parameters(),lr=0.0001)
discriminator=Discriminator()
d_optimizer=torch.optim.Adam(discriminator.parameters(),lr=0.0001)
loss=nn.BCELoss()

In [57]:
num_epoch=10
latent_dim=96
i=0
for epoch in range(num_epoch):
    print(f"第{epoch+1}轮训练!")
    
    for index,minibatch in enumerate(dataloader):
        #生成器优化
        i=index
        true_image,_=minibatch
        z=torch.randn(batchsize,latent_dim)
        pre_img=generator(z)
        g_optimizer.zero_grad()
        g_loss=loss(discriminator(pre_img),torch.ones(batchsize,1))
        g_loss.backward()
        g_optimizer.step()

        #判别器优化
        d_optimizer.zero_grad()
        # d_loss=0.5*(loss(discriminator(true_image),torch.ones(batchsize,1))+loss(discriminator(pre_img.detach()),torch.zeros(batchsize,1)))
        real_loss=loss(discriminator(true_image),torch.ones(batchsize,1))
        fake_loss=loss(discriminator(pre_img.detach()),torch.zeros(batchsize,1))
        d_loss=real_loss*0.5+fake_loss*0.5
        d_loss.backward()
        d_optimizer.step()
        # if i % 50 == 0:
            # print(f"step:{len(dataloader)*epoch+i}, g_loss:{g_loss.item()}, d_loss:{d_loss.item()}, real_loss:{real_loss.item()}, fake_loss:{fake_loss.item()}")

        if index%10000==0:
            for index,img in enumerate(pre_img):
                torchvision.utils.save_image(img,f"img/img_{index}_{epoch}.png")
        # break
    # break




第1轮训练!
第2轮训练!
第3轮训练!
第4轮训练!
第5轮训练!
第6轮训练!
第7轮训练!
第8轮训练!
第9轮训练!
第10轮训练!
